# Speed Skydiving Analysis and Scoring

## Environment setup

In [8]:
from ssscore import COURSE_START
from ssscore import COURSE_END
from ssscore import DEG_IN_RAD
from ssscore import VALID_MSL
from ssscore import YEAR_PATH_TAG

import dateutil.parser
import math
import os
import os.path
import shutil

import pandas as pd

import ssscore

## FlySight data sources

1. Copy the FlySight Tracks/YY-MM-dd directory of interest to the DATA_LAKE directory;
   the DATA_LAKE can also be an external mount, a Box or Dropbox share, anything that 
   can be mapped to a directory -- even a whole drive!
1. Make a list of all CSV files in the FlySight DATA_LAKE
1. Move the CSV to the DATA_SOURCE bucket directory

### Define the data lake and data source

![Data sources diagram](images/SSScoring-data-sources.png)
<a id="l_data-def"></a>

In [ ]:
DATA_LAKE   = './data-lake'
DATA_SOURCE = './data-sources/pr3d4t0r'

ssscore.updateFlySightDataSource(DATA_LAKE, DATA_SOURCE)

## Top speed and pitch analysis on a single data file

User selects a valid FlySight data path and file name.  Call the `calculateSpeedAndPitch(fileName)` function with this file name.  The function produces:

* Classical mean speed calculated from all the discrete speeds sampled within the course
* USPA calculation ds/dt where dt<sub>competitionTime</sub> = t<sub>end</sub>-t<sub>start</sub>,
  and s<sub>courseLength</sub> = |s<sub>start</sub>-s<sub>end</sub>| for the first and last heights
  reported within the course window.
* Flight pitch in degrees at max speed
* Max speed
* Min speed

All speeds are reported in km/h.  Michael Cooper, USPA National Speed Skydiving Championship judged, explained
the "USPA calculation" as straight ds/dt in an email message from 04.Sep.2018.

### Explanation of the code in calculateSpeedAndPitchFrom(fileName)

1. Discard all source entries outside of the 2,700 to 1,700 m AGL course
1. Calculate the mean speed for the current jump (actual sum(N)/N and ds/dt for s0, s1
1. Resolve the max speed and pitch within the course
1. Return the results in a Series object, for later inclusion in a data frame with results from multiple jumps

### Specify the FlySight data file to analyze

In [18]:
DATA_SOURCE        = './data-sources/ciurana'
FLYSIGHT_DATA_FILE = '20181112.122617.CSV'

In [19]:
def _discardDataOutsideCourse(fullFlightData):
    height          = fullFlightData['hMSL', '(m)']
    descentVelocity = fullFlightData['velD', '(m/s)']
    
    return fullFlightData[(height <= COURSE_START) & (height >= COURSE_END) & (descentVelocity >= 0.0)]


def _calculateCourseSpeedUsing(flightData):
    """
    Returns course speed as ds/dt, and competitionTime in seconds
    """
    # TODO: figure out why pd.to_datetime() barfs
    startTime = dateutil.parser.parse(flightData['time', 'Unnamed: 0_level_1'].values[0])
    endTime   = dateutil.parser.parse(flightData['time', 'Unnamed: 0_level_1'].values[-1])

    startCourse = flightData['hMSL', '(m)'].values[0]
    endCourse   = flightData['hMSL', '(m)'].values[-1]

    competitionTime = (endTime-startTime).total_seconds()
    courseLength    = abs(endCourse-startCourse)

    return courseLength/competitionTime, competitionTime


def maxHorizontalSpeedFrom(flightData, maxVerticalSpeed):
    velN = flightData[flightData['velD', '(m/s)'] == maxVerticalSpeed]['velN', '(m/s)']
    velE = flightData[flightData['velD', '(m/s)'] == maxVerticalSpeed]['velE', '(m/s)']
    
    return math.sqrt(velN**2+velE**2)   # R vector


def calculateSpeedAndPitchFor(fileName):
    """
    Accepts a file name to a FlySight data file.
    
    Returns a Series with the results of a speed skydiving jump.
    """
    flightData  = _discardDataOutsideCourse(pd.read_csv(fileName, header = [0, 1]))
    sampledVelD = flightData['velD', '(m/s)'].mean()
    courseVelD, \
    courseTime  = _calculateCourseSpeedUsing(flightData)
    maxSpeed    = flightData['velD', '(m/s)'].max()
    minSpeed    = flightData['velD', '(m/s)'].min()
    
    pitchR       = math.atan(maxSpeed/maxHorizontalSpeedFrom(flightData, maxSpeed))
    maxSpeedTime = flightData[flightData['velD', '(m/s)'] == maxSpeed]['time', 'Unnamed: 0_level_1'].values[0]
    
    skydiveResults = pd.Series(
                        [
                            fileName,
                            maxSpeedTime,
                            3.6*sampledVelD,  # km/h; 3,600 seconds, 1,000 meters
                            3.6*courseVelD,
                            pitchR/DEG_IN_RAD,
                            3.6*maxSpeed,
                            3.6*minSpeed,
                            courseTime,
                        ],
                        [
                            'fileName',
                            'maxSpeedTime',
                            'sampledSpeed',
                            'courseSpeed',
                            'pitch',
                            'maxSpeed',
                            'minSpeed',
                            'courseTime',
                        ])

    return skydiveResults


calculateSpeedAndPitchFor(os.path.join(DATA_SOURCE, FLYSIGHT_DATA_FILE))

fileName        ./data-sources/ciurana/20181112.122617.CSV
maxSpeedTime                       2018-11-12T19:45:12.80Z
sampledSpeed                                       405.254
courseSpeed                                        405.769
pitch                                              72.0859
maxSpeed                                           415.116
minSpeed                                           397.836
courseTime                                             8.6
dtype: object

## Listing FlySight data files with valid data

This set up generates a list of FlySight data files ready for analysis.  It discards any warm up FlySight data files, those that show no significant changes in elevation MSL across the complete data set.

The list generator uses the `DATA_SOURCE` global variable.  [Change the value of `DATA_SOURCE`](#l_data-def) if necessary.

1. Generate the list of available data files
1. Discard FlySight warm up files by rejecting files without jump data (test:  minimal altitude changes)

![Scoring FlySight files list generation diagram](images/SSScoring-list-scoring-files.png)

In [20]:
def listDataFilesIn(bucketPath):
    """
    Generate a sorted list of files available in a given bucketPath.
    Files names appear in reverse lexicographical order.
    """

    filesList = pd.Series([os.path.join(bucketPath, fileName) for fileName in sorted(os.listdir(bucketPath), reverse = True) if '.CSV' in fileName])
    
    return filesList;

In [21]:
def _hasValidJumpData(fileName):
    flightData = pd.read_csv(fileName, header = [0, 1])['hMSL', '(m)']
    
    return flightData.std() >= VALID_MSL

In [22]:
def selectValidFlySightFilesFrom(dataFiles):
    included = dataFiles.apply(_hasValidJumpData)
    
    return pd.Series(dataFiles)[included]

In [23]:
dataFiles = selectValidFlySightFilesFrom(listDataFilesIn(DATA_SOURCE))
dataFiles

0    ./data-sources/ciurana/20181112.131849.CSV
1    ./data-sources/ciurana/20181112.122617.CSV
2    ./data-sources/ciurana/20181112.111218.CSV
dtype: object

## Top speed and pitch analysis on all the data in a bucket

Takes all the FlySight files in a bucket, detects the ones with valid data, and runs performance analysis over them.  Packs all the results in a data frame, then calculates:

* Average speed
* Max average speed

_Future versions may adhere to USPA / ISSA judging rules for analysis._

### Define the data source

In [24]:
DATA_SOURCE = './data-sources/ciurana'

### Analyze all files in the bucket

In [25]:
allCompetitionJumps = selectValidFlySightFilesFrom(listDataFilesIn(DATA_SOURCE)).apply(calculateSpeedAndPitchFor)
allCompetitionJumps

,fileName,maxSpeedTime,sampledSpeed,courseSpeed,pitch,maxSpeed,minSpeed,courseTime
0,./data-sources/ciurana/20181112.131849.CSV,2018-11-12T20:36:01.40Z,313.522759,311.750211,69.907863,398.736,242.712,11.4
1,./data-sources/ciurana/20181112.122617.CSV,2018-11-12T19:45:12.80Z,405.253636,405.768558,72.085916,415.116,397.836,8.6
2,./data-sources/ciurana/20181112.111218.CSV,2018-11-12T18:35:10.40Z,328.294667,329.788528,66.859584,356.580,283.140,10.6


### Summary of results

In [26]:
def competitionResults(jumpsData):
    meanCourseSpeed  = jumpsData['courseSpeed'].mean()
    meanSampledSpeed = jumpsData['sampledSpeed'].mean()
    meanPitch        = jumpsData['pitch'].mean()
    meanTime         = jumpsData['courseTime'].mean()
    maxSpeed         = jumpsData['courseSpeed'].max()

    return pd.Series(
        [
            meanCourseSpeed,
            meanSampledSpeed,
            meanPitch,
            meanTime,
            maxSpeed,
        ],
        [
            'meanCourseSpeed',
            'meanSampledSpeed',
            'meanPitch',
            'meanTime',
            'maxSpeed',
        ])

competitionResults(allCompetitionJumps)

meanCourseSpeed     349.102432
meanSampledSpeed    349.023687
meanPitch            69.617788
meanTime             10.200000
maxSpeed            405.768558
dtype: float64